# Topic 29: Machine Learning Basics with Scikit-Learn

## Overview
Scikit-learn is Python's most popular machine learning library, providing simple and efficient tools for data mining and analysis[9][25]. It offers a consistent API for various ML algorithms and preprocessing tools.

### What You'll Learn:
- Machine learning workflow and concepts
- Supervised learning: classification and regression
- Unsupervised learning: clustering and dimensionality reduction
- Model evaluation and validation
- Data preprocessing and feature engineering
- Pipeline creation and model selection

---

## 1. Machine Learning Fundamentals

Understanding ML concepts and the scikit-learn API:

In [ ]:
# Machine Learning Fundamentals with Scikit-Learn
from sklearn import datasets, model_selection, preprocessing, metrics
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Machine Learning Fundamentals with Scikit-Learn:")
print("=" * 47)

# Machine Learning Problem Types
print("1. Machine Learning Problem Types:")
ml_types = {
    'Supervised Learning': {
        'Classification': 'Predict categories (spam/not spam, disease/healthy)',
        'Regression': 'Predict continuous values (price, temperature, sales)'
    },
    'Unsupervised Learning': {
        'Clustering': 'Group similar data points (customer segments)',
        'Dimensionality Reduction': 'Reduce feature space (visualization, compression)'
    },
    'Semi-supervised Learning': {
        'Mixed': 'Uses both labeled and unlabeled data'
    },
    'Reinforcement Learning': {
        'Sequential': 'Learn through rewards and penalties (games, robotics)'
    }
}

for category, problems in ml_types.items():
    print(f"   {category}:")
    for problem_type, description in problems.items():
        print(f"     {problem_type}: {description}")

# Load sample datasets
print("\n2. Loading sample datasets:")

# Classification dataset (Iris)
iris = datasets.load_iris()
print(f"   Iris dataset: {iris.data.shape} features, {len(iris.target_names)} classes")
print(f"   Features: {iris.feature_names}")
print(f"   Classes: {iris.target_names}")

# Regression dataset (Boston Housing - if available, otherwise California Housing)
try:
    housing = datasets.load_boston()
    print(f"   Boston Housing: {housing.data.shape} features, continuous target")
except:
    housing = datasets.fetch_california_housing()
    print(f"   California Housing: {housing.data.shape} features, continuous target")

# Convert to DataFrames for easier handling
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df['species'] = [iris.target_names[i] for i in iris.target]

housing_df = pd.DataFrame(housing.data, columns=housing.feature_names)
housing_df['target'] = housing.target

print(f"\n3. Exploring the datasets:")
print(f"   Iris dataset shape: {iris_df.shape}")
print(f"{iris_df.head()}")

print(f"\n   Housing dataset (first 5 columns): {housing_df.iloc[:, :5].shape}")
print(f"{housing_df.iloc[:, :5].head()}")

# Basic statistics
print(f"\n4. Dataset statistics:")
print(f"   Iris target distribution:")
print(f"{iris_df['species'].value_counts()}")
print(f"\n   Housing target statistics:")
print(f"     Mean price: ${housing_df['target'].mean():.2f}")
print(f"     Std deviation: ${housing_df['target'].std():.2f}")
print(f"     Price range: ${housing_df['target'].min():.2f} - ${housing_df['target'].max():.2f}")

# Scikit-learn API pattern
print(f"\n5. Scikit-learn API pattern:")
print(f"   1. Import the model class")
print(f"   2. Instantiate the model: model = ModelClass(parameters)")
print(f"   3. Fit the model: model.fit(X_train, y_train)")
print(f"   4. Make predictions: predictions = model.predict(X_test)")
print(f"   5. Evaluate: score = model.score(X_test, y_test)")

# Demonstrate basic workflow
print(f"\n6. Basic ML workflow demonstration:")

# Split the iris data
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"   Dataset split:")
print(f"   Training set: {X_train.shape}")
print(f"   Test set: {X_test.shape}")

# Train a simple classifier
clf = LogisticRegression(random_state=42, max_iter=200)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)
accuracy = clf.score(X_test, y_test)

print(f"   Model trained and evaluated:")
print(f"   Accuracy: {accuracy:.3f}")
print(f"   Predictions: {y_pred[:10]}")
print(f"   Actual: {y_test[:10]}")

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Feature visualization
sns.scatterplot(data=iris_df, x='sepal length (cm)', y='sepal width (cm)', 
               hue='species', ax=axes[0])
axes[0].set_title('Iris Dataset: Sepal Dimensions')

# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=iris.target_names, 
           yticklabels=iris.target_names, ax=axes[1])
axes[1].set_title('Confusion Matrix')
axes[1].set_ylabel('Actual')
axes[1].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

print(f"   ✓ Successfully completed basic ML workflow")

## 2. Supervised Learning: Classification

Building and comparing classification models:

In [ ]:
# Supervised Learning: Classification
print("Supervised Learning: Classification:")
print("=" * 34)

# Prepare data
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features (important for some algorithms)
scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("1. Multiple classification algorithms:")

# Define classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=200),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
}

# Train and evaluate each classifier
results = []

for name, clf in classifiers.items():
    # Use scaled data for SVM and KNN, original for tree-based methods
    if name in ['SVM (RBF)', 'K-Nearest Neighbors', 'Logistic Regression']:
        clf.fit(X_train_scaled, y_train)
        y_pred = clf.predict(X_test_scaled)
        y_pred_proba = clf.predict_proba(X_test_scaled) if hasattr(clf, 'predict_proba') else None
    else:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test) if hasattr(clf, 'predict_proba') else None
    
    # Calculate metrics
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1 = metrics.f1_score(y_test, y_pred, average='weighted')
    
    results.append({
        'Algorithm': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"   {name}:")
    print(f"     Accuracy: {accuracy:.3f}")
    print(f"     Precision: {precision:.3f}")
    print(f"     Recall: {recall:.3f}")
    print(f"     F1-Score: {f1:.3f}")

# Convert results to DataFrame for visualization
results_df = pd.DataFrame(results)

# Visualize algorithm comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Classification Algorithm Comparison', fontsize=16, fontweight='bold')

# Metrics comparison
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
for i, metric in enumerate(metrics_to_plot):
    row, col = i // 2, i % 2
    sns.barplot(data=results_df, x='Algorithm', y=metric, ax=axes[row, col])
    axes[row, col].set_title(f'{metric} Comparison')
    axes[row, col].set_xticklabels(axes[row, col].get_xticklabels(), rotation=45)
    axes[row, col].set_ylim(0, 1)

plt.tight_layout()
plt.show()

print(f"\n2. Detailed classification report:")

# Use the best performing classifier for detailed analysis
best_clf = RandomForestClassifier(n_estimators=100, random_state=42)
best_clf.fit(X_train, y_train)
y_pred_best = best_clf.predict(X_test)

# Classification report
print(f"   Classification Report (Random Forest):")
print(metrics.classification_report(y_test, y_pred_best, 
                                  target_names=iris.target_names))

# Feature importance
feature_importance = best_clf.feature_importances_
feature_names = iris.feature_names

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Feature importance plot
feature_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=True)

sns.barplot(data=feature_df, x='importance', y='feature', ax=ax1)
ax1.set_title('Feature Importance (Random Forest)')
ax1.set_xlabel('Importance Score')

# Confusion matrix heatmap
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=iris.target_names,
           yticklabels=iris.target_names, ax=ax2)
ax2.set_title('Confusion Matrix')
ax2.set_ylabel('Actual')
ax2.set_xlabel('Predicted')

plt.tight_layout()
plt.show()

print(f"   ✓ Feature importance analysis completed")

print(f"\n3. Cross-validation for robust evaluation:")

# Perform cross-validation
cv_scores = cross_val_score(best_clf, X, y, cv=5, scoring='accuracy')

print(f"   5-Fold Cross-Validation Results:")
print(f"   Individual fold scores: {cv_scores}")
print(f"   Mean accuracy: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")
print(f"   This gives us confidence in model performance")

print(f"\n4. Hyperparameter tuning with Grid Search:")

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10]
}

# Perform grid search
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), 
                          param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f"   Best parameters: {grid_search.best_params_}")
print(f"   Best cross-validation score: {grid_search.best_score_:.3f}")
print(f"   Test set accuracy: {grid_search.score(X_test, y_test):.3f}")

print(f"\n5. Model interpretation and predictions:")

# Make predictions on new data
new_samples = np.array([[5.1, 3.5, 1.4, 0.2],  # Likely Setosa
                       [6.2, 2.8, 4.8, 1.8],  # Likely Versicolor 
                       [7.2, 3.0, 5.8, 2.2]]) # Likely Virginica

predictions = best_clf.predict(new_samples)
prediction_proba = best_clf.predict_proba(new_samples)

print(f"   New sample predictions:")
for i, (sample, pred, proba) in enumerate(zip(new_samples, predictions, prediction_proba)):
    species_name = iris.target_names[pred]
    confidence = max(proba)
    print(f"   Sample {i+1}: {sample}")
    print(f"     Predicted: {species_name} (confidence: {confidence:.3f})")
    print(f"     Class probabilities: {dict(zip(iris.target_names, proba.round(3)))}")

print(f"\n6. Classification best practices:")
print(f"   ✓ Always split data into train/validation/test sets")
print(f"   ✓ Scale features for distance-based algorithms")
print(f"   ✓ Use cross-validation for robust evaluation")
print(f"   ✓ Consider class imbalance in real datasets")
print(f"   ✓ Interpret feature importance and model decisions")
print(f"   ✓ Tune hyperparameters systematically")
print(f"   ✓ Evaluate using multiple metrics (precision, recall, F1)")

## 3. Supervised Learning: Regression

Building regression models for continuous predictions:

In [ ]:
# Supervised Learning: Regression
print("Supervised Learning: Regression:")
print("=" * 30)

# Prepare regression data
X_reg = housing.data
y_reg = housing.target

# Split the data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42)

# Scale features
scaler_reg = preprocessing.StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

print(f"1. Regression dataset overview:")
print(f"   Features: {X_reg.shape[1]}")
print(f"   Samples: {X_reg.shape[0]}")
print(f"   Target range: {y_reg.min():.2f} - {y_reg.max():.2f}")
print(f"   Training set: {X_train_reg.shape}")
print(f"   Test set: {X_test_reg.shape}")

print(f"\n2. Multiple regression algorithms:")

# Define regressors
regressors = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'SVM (RBF)': SVR(kernel='rbf'),
}

# Train and evaluate each regressor
regression_results = []

for name, reg in regressors.items():
    # Use scaled data for SVM, original for tree-based methods
    if name in ['SVM (RBF)', 'Linear Regression']:
        reg.fit(X_train_reg_scaled, y_train_reg)
        y_pred_reg = reg.predict(X_test_reg_scaled)
    else:
        reg.fit(X_train_reg, y_train_reg)
        y_pred_reg = reg.predict(X_test_reg)
    
    # Calculate regression metrics
    mse = metrics.mean_squared_error(y_test_reg, y_pred_reg)
    rmse = np.sqrt(mse)
    mae = metrics.mean_absolute_error(y_test_reg, y_pred_reg)
    r2 = metrics.r2_score(y_test_reg, y_pred_reg)
    
    regression_results.append({
        'Algorithm': name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    })
    
    print(f"   {name}:")
    print(f"     MSE: {mse:.3f}")
    print(f"     RMSE: {rmse:.3f}")
    print(f"     MAE: {mae:.3f}")
    print(f"     R² Score: {r2:.3f}")

# Convert results to DataFrame
regression_results_df = pd.DataFrame(regression_results)

# Visualize regression results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Regression Algorithm Comparison', fontsize=16, fontweight='bold')

# Metrics comparison
regression_metrics = ['MSE', 'RMSE', 'MAE', 'R²']
for i, metric in enumerate(regression_metrics):
    row, col = i // 2, i % 2
    sns.barplot(data=regression_results_df, x='Algorithm', y=metric, ax=axes[row, col])
    axes[row, col].set_title(f'{metric} Comparison')
    axes[row, col].set_xticklabels(axes[row, col].get_xticklabels(), rotation=45)
    
    # Set y-limit for R² to show differences better
    if metric == 'R²':
        axes[row, col].set_ylim(0, 1)

plt.tight_layout()
plt.show()

print(f"\n3. Detailed analysis of best model:")

# Use Random Forest as the best model (typically performs well)
best_reg = RandomForestRegressor(n_estimators=100, random_state=42)
best_reg.fit(X_train_reg, y_train_reg)
y_pred_best_reg = best_reg.predict(X_test_reg)

# Feature importance for regression
feature_importance_reg = best_reg.feature_importances_
feature_names_reg = housing.feature_names

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Regression Model Analysis', fontsize=16, fontweight='bold')

# Feature importance
feature_df_reg = pd.DataFrame({
    'feature': feature_names_reg,
    'importance': feature_importance_reg
}).sort_values('importance', ascending=True)

sns.barplot(data=feature_df_reg.tail(10), x='importance', y='feature', ax=axes[0,0])
axes[0,0].set_title('Top 10 Feature Importance')
axes[0,0].set_xlabel('Importance Score')

# Predictions vs Actual
axes[0,1].scatter(y_test_reg, y_pred_best_reg, alpha=0.5)
axes[0,1].plot([y_test_reg.min(), y_test_reg.max()], 
              [y_test_reg.min(), y_test_reg.max()], 'r--', lw=2)
axes[0,1].set_xlabel('Actual Values')
axes[0,1].set_ylabel('Predicted Values')
axes[0,1].set_title('Predictions vs Actual Values')

# Residual plot
residuals = y_test_reg - y_pred_best_reg
axes[1,0].scatter(y_pred_best_reg, residuals, alpha=0.5)
axes[1,0].axhline(y=0, color='r', linestyle='--')
axes[1,0].set_xlabel('Predicted Values')
axes[1,0].set_ylabel('Residuals')
axes[1,0].set_title('Residual Plot')

# Residual distribution
axes[1,1].hist(residuals, bins=30, alpha=0.7)
axes[1,1].set_xlabel('Residuals')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title('Residual Distribution')

plt.tight_layout()
plt.show()

print(f"   ✓ Feature importance and residual analysis completed")

print(f"\n4. Model validation and learning curves:")

# Learning curve analysis
from sklearn.model_selection import learning_curve

train_sizes, train_scores, val_scores = learning_curve(
    best_reg, X_train_reg, y_train_reg, 
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5, scoring='r2'
)

# Plot learning curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Learning curve
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

ax1.plot(train_sizes, train_mean, 'o-', label='Training Score', color='blue')
ax1.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='blue')
ax1.plot(train_sizes, val_mean, 'o-', label='Validation Score', color='red')
ax1.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color='red')
ax1.set_xlabel('Training Set Size')
ax1.set_ylabel('R² Score')
ax1.set_title('Learning Curve')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cross-validation scores
cv_scores_reg = cross_val_score(best_reg, X_reg, y_reg, cv=5, scoring='r2')
ax2.bar(range(1, 6), cv_scores_reg)
ax2.axhline(y=cv_scores_reg.mean(), color='red', linestyle='--', 
           label=f'Mean: {cv_scores_reg.mean():.3f}')
ax2.set_xlabel('Fold')
ax2.set_ylabel('R² Score')
ax2.set_title('5-Fold Cross-Validation Scores')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"   Cross-validation R² scores: {cv_scores_reg}")
print(f"   Mean CV R² score: {cv_scores_reg.mean():.3f} (+/- {cv_scores_reg.std() * 2:.3f})")

print(f"\n5. Making predictions on new data:")

# Example predictions
# Create some example house features (scaled appropriately)
new_houses = np.array([
    X_reg[0],  # Use first house as example
    X_reg[100], # Use 100th house as example
    np.mean(X_reg, axis=0)  # Average house
])

predictions_reg = best_reg.predict(new_houses)

print(f"   Example house predictions:")
for i, (house_features, predicted_price) in enumerate(zip(new_houses, predictions_reg)):
    print(f"   House {i+1}:")
    print(f"     Predicted price: ${predicted_price:.2f}")
    print(f"     Features: {house_features[:3].round(2)}... (showing first 3)")

print(f"\n6. Regression model interpretation:")

# For Linear Regression, show coefficients
linear_reg = LinearRegression()
linear_reg.fit(X_train_reg_scaled, y_train_reg)

coefficients = pd.DataFrame({
    'Feature': feature_names_reg,
    'Coefficient': linear_reg.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(f"   Linear Regression Coefficients (top 5):")
for _, row in coefficients.head().iterrows():
    print(f"     {row['Feature']}: {row['Coefficient']:.3f}")

print(f"\n7. Regression best practices:")
print(f"   ✓ Check for linear relationships with scatter plots")
print(f"   ✓ Handle outliers that can skew results")
print(f"   ✓ Scale features for linear models")
print(f"   ✓ Use multiple evaluation metrics (R², RMSE, MAE)")
print(f"   ✓ Analyze residuals for model diagnostics")
print(f"   ✓ Use cross-validation for robust evaluation")
print(f"   ✓ Consider feature engineering and polynomial terms")

## Summary

In this notebook, you learned about:

✅ **ML Fundamentals**: Problem types, supervised vs unsupervised learning[9][22]  
✅ **Scikit-learn API**: Consistent interface for all algorithms[25]  
✅ **Classification**: Multiple algorithms, evaluation metrics, hyperparameter tuning[12]  
✅ **Regression**: Continuous prediction, residual analysis, feature importance  
✅ **Model Validation**: Cross-validation, learning curves, robust evaluation  
✅ **Best Practices**: Data splitting, preprocessing, interpretation[22]  

### Key Takeaways:
1. Scikit-learn provides a unified API for diverse ML algorithms[9][25]
2. Always split data and use cross-validation for reliable results[12]
3. Feature scaling is crucial for distance-based algorithms
4. Multiple metrics provide better model understanding than single scores
5. Hyperparameter tuning can significantly improve performance[22]
6. Model interpretation is as important as performance metrics

### Next Topic: 30_web_scraping.ipynb
Learn how to collect data from websites using Python scraping libraries.